In [1]:
# Full setup
from dataclasses import dataclass, field

from typing import List
from datetime import datetime

## init & repr
* Created by default

In [2]:
# Nothing specified
@dataclass
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)
print(car)
car = Car('honda', 'civic', 1998)
print(car)

Car(make='chevy', model='sonic', year=2016)
Car(make='honda', model='civic', year=1998)


In [3]:
# Without typehints you get an error
@dataclass
class Car:
    make
    model
    year

NameError: name 'make' is not defined

In [4]:
# init=False
@dataclass(init=False)
class Car:
    make: str
    model: str
    year: int

Car(make='chevy', model='sonic', year=2016)

TypeError: Car() takes no arguments

In [5]:
# __init__ is defined (or any other dunder method)
@dataclass(init=True)
class Car:
    make: str
    model: str
    year: int
        
    def __init__(self):
        pass

Car(make='chevy', model='sonic', year=2016)

TypeError: __init__() got an unexpected keyword argument 'make'

In [6]:
# repr is False
@dataclass(repr=False)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)
print(car)

In [7]:
# When a class variable is set, it is a default argument in __init__
@dataclass
class Car:
    make: str
    model: str
    year: int = 2019
        
car = Car(make='chevy', model='sonic')
print(car)
car = Car(make='chevy', model='sonice', year=2016)
print(car)

Car(make='chevy', model='sonic', year=2019)
Car(make='chevy', model='sonice', year=2016)


In [8]:
# Order matters when setting class variables
@dataclass
class Car:
    make: str = 'chevy'
    model: str
    year: int
        
car = Car(model='sonic', year=2016)
print(car)

TypeError: non-default argument 'model' follows default argument

In [9]:
# Required order if settings defaults
@dataclass
class Car:
    model: str
    year: int
    make: str = 'chevy'

car = Car(model='sonic', year=2016)
print(car)

Car(model='sonic', year=2016, make='chevy')


# comparisons
* Creates equals by default
* Does not create gt, gte, lt, lte by default

In [10]:
# Default comparisons
@dataclass
class Car:
    make: str
    model: str
    year: int
        
car1 = Car(make='chevy', model='sonic', year=2016)
car2 = Car(make='chevy', model='sonic', year=2016)
car3 = Car(make='chevy', model='sonic', year=2015)
print(car1 == car2)
print(car1 == car3)
car1 < car3

True
False


TypeError: '<' not supported between instances of 'Car' and 'Car'

In [11]:
# Try to include order without equals
@dataclass(order=True, eq=False)
class Car:
    make: str
    model: str
    year: int
        
car1 = Car(make='chevy', model='sonic', year=2016)
car2 = Car(make='chevy', model='sonic', year=2016)
car3 = Car(make='chevy', model='sonic', year=2015)
print(car1 == car2)
print(car1 == car3)
car1 < car3

ValueError: eq must be true if order is true

In [16]:
# Set order to True
@dataclass(order=True)
class Car:
    make: str
    model: str
    year: int
        
car1 = Car(make='chevy', model='sonic', year=2016)
car2 = Car(make='audi', model='sonic', year=2016)
car3 = Car(make='chevy', model='sonic', year=2015)
print(car1 < car3)
print(car1 > car2)
print(car1 > car3)
cars = [car1, car3, car2]
for car in cars:
    print(car)
ordered_cars = list(sorted(cars))
for car in ordered_cars:
    print(car)

False
True
True
Car(make='chevy', model='sonic', year=2016)
Car(make='chevy', model='sonic', year=2015)
Car(make='audi', model='sonic', year=2016)
Car(make='audi', model='sonic', year=2016)
Car(make='chevy', model='sonic', year=2015)
Car(make='chevy', model='sonic', year=2016)


In [19]:
# Set order to True, but define a comparison method
@dataclass(order=True)
class Car:
    make: str
    model: str
    year: int
    
    def __lt__(self, other: 'Car'):
        return True

TypeError: Cannot overwrite attribute __lt__ in class Car. Consider using functools.total_ordering

## Freezing a dataclass
* It is possible to make a dataclass that cannot be edited

In [20]:
# data can be changed by default
@dataclass
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)
car.make = 'dodge'
print(car)

Car(make='dodge', model='sonic', year=2016)


In [25]:
# frozen=True makes it imposssible to change a field
@dataclass(frozen=True)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)
car.make = 'ford'

FrozenInstanceError: cannot assign to field 'make'

In [26]:
# FrozenInstanceError is a special error specific to dataclasses
# frozen=True makes it imposssible to delete a field
@dataclass(frozen=True)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)
delattr(car, 'make')

FrozenInstanceError: cannot delete field 'make'

## hashing
* If a dataclass is determined to be safely hashable, then a `__hash__` method is automatically created
* If you want to allow a dataclass to be hashed even if it is not safe

In [27]:
# Try using a dataclass instance as a key to a dictionary (requires __hash__)
@dataclass
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)

cars = {car: 32}

TypeError: unhashable type: 'Car'

In [28]:
# setting unsafe_hash=True means that __hash__ will be created no matter what
@dataclass(unsafe_hash=True)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)

cars = {car: 32}
print(cars)
print(cars[car])

# Will work for any instance of car created with those arguments
print(cars[Car(make='chevy', model='sonic', year=2016)])

{Car(make='chevy', model='sonic', year=2016): 32}
32
32


In [31]:
# setting unsafe_hash=True means you could run into
# problems if you edit your instance
@dataclass(unsafe_hash=True)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)

cars = {car: 32}
print(cars)
print(cars[car])

car.make = 'ford'
print(cars)
print(cars[car])
print(cars[Car(make='chevy', model='sonic', year=2016)])

{Car(make='chevy', model='sonic', year=2016): 32}
32
{Car(make='ford', model='sonic', year=2016): 32}
32


KeyError: Car(make='chevy', model='sonic', year=2016)

In [33]:
# __hash__ will be created automatically if frozen=True,
# but if eq=False then __hash__ is based on the instance id
@dataclass(frozen=True, eq=False)
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)

cars = {car: 32}
print(cars)
print(cars[car])
print(car.__hash__())
print(cars[Car(make='chevy', model='sonic', year=2016)])

{Car(make='chevy', model='sonic', year=2016): 32}
32
275905353


KeyError: Car(make='chevy', model='sonic', year=2016)

In [34]:
# if both frozen=True and eq=True then hash is created for all variables by default
@dataclass(frozen=True, eq=True)  # eq=True is the default, just using it here to be explicit
class Car:
    make: str
    model: str
    year: int
        
car = Car(make='chevy', model='sonic', year=2016)

cars = {car: 32}
print(cars)
print(cars[car])
print(cars[Car(make='chevy', model='sonic', year=2016)])

{Car(make='chevy', model='sonic', year=2016): 32}
32
32


## Customization
* What if you don't want to include a variable in equals or the hash
* What if you want a special default argument

In [35]:
# exclude a field from everything (and provide a default_factory)
@dataclass
class Car:
    make: str
    model: str
    year: int
    owners: List[str] = field(
        default_factory=list, repr=False, init=False,
        compare=False, hash=False,
    )

car = Car(make='chevy', model='sonic', year=2016)
print(car)
print(car.owners)

Car(make='chevy', model='sonic', year=2016)
[]


In [36]:
# if init=False on a field, it can't be used in init
@dataclass
class Car:
    make: str
    model: str
    year: int
    owners: List[str] = field(default_factory=list, init=False)

car = Car(make='chevy', model='sonic', year=2016, owners=['me'])

TypeError: __init__() got an unexpected keyword argument 'owners'

In [37]:
# if compare=False on a field, it won't be used in comparisons
@dataclass
class Car:
    make: str
    model: str
    year: int
    owners: List[str] = field(compare=False)

car1 = Car(make='chevy', model='sonic', year=2016, owners=['me'])
print(car1.owners)
car2 = Car(make='chevy', model='sonic', year=2016, owners=['you'])
print(car2.owners)
print(car1 == car2)

['me']
['you']
True


In [39]:
# default_factory on a field can be any callable
@dataclass
class Car:
    make: str
    model: str
    year: int
    last_service: datetime = field(default_factory=datetime.now)
        
car = Car(make='chevy', model='sonic', year=2016)
print(car)

Car(make='chevy', model='sonic', year=2016, last_service=datetime.datetime(2019, 8, 13, 19, 22, 49, 396438))


In [44]:
# a default can also be specified on a field 
# since you can't use `field` and set a default the normal way
@dataclass
class Car:
    make: str
    model: str
    year: int
    production_date: datetime = field(
        default=datetime(2019, 1, 1),
        compare=False,
    )
        
car = Car(make='chevy', model='sonic', year=2016)
print(car)

Car(make='chevy', model='sonic', year=2016, production_date=datetime.datetime(2019, 1, 1, 0, 0))


In [45]:
# Can't use default with mutable types
@dataclass
class Car:
    make: str
    model: str
    year: int
    owners: List[str] = field(default=[])

ValueError: mutable default <class 'list'> for field owners is not allowed: use default_factory

In [46]:
# Can't set a default value "normally" if you have a mutable type
@dataclass
class Car:
    make: str
    model: str
    year: int
    owners: List[str] = []

ValueError: mutable default <class 'list'> for field owners is not allowed: use default_factory

## Other useful hints
* Inheritance
* Class attributes
* Do extra things after `__init__`

In [47]:
# Inheriting from a normal class
class Animal:
    has_fur: bool
        
@dataclass
class Cat(Animal):
    name: str

cat = Cat(name='Felix')
print(cat)
cat = Cat(has_fur=True, name='Felix')

Cat(name='Felix')


TypeError: __init__() got an unexpected keyword argument 'has_fur'

In [48]:
# If you inherit from a dataclass, you get anything set on the other dataclass for free
@dataclass
class Animal:
    has_fur: bool
        
@dataclass
class Cat(Animal):
    name: str

cat = Cat(has_fur=True, name='Felix')
print(cat)

Cat(has_fur=True, name='Felix')


In [49]:
# fields without defaults are not class variables
@dataclass
class Car:
    make: str
    model: str
    year: int

car = Car(make='chevy', model='sonic', year=2016)
print(car)
print(Car.make)

Car(make='chevy', model='sonic', year=2016)


AttributeError: type object 'Car' has no attribute 'make'

In [50]:
# Fields with default_factory set are also not class variables (even though a "value" is set on them)
@dataclass
class Car:
    make: str
    model: str
    year: int
    last_service: datetime = field(default_factory=datetime.now)
    production_date: datetime = field(default=datetime(2019, 1, 1))

car = Car(make='chevy', model='sonic', year=2016)
print(Car.last_service)

AttributeError: type object 'Car' has no attribute 'last_service'

In [53]:
# A "normal" field with a default or a dataclass.field
# with a default argument are both set on the class
@dataclass
class Car:
    make: str
    model: str
    year: int = 2019
    production_date: datetime = field(default=datetime(2019, 1, 1))
    

car = Car(make='chevy', model='sonic', year=2016, production_date=datetime(2018, 1, 1))
print(Car.year)
print(Car.production_date)

2019
2019-01-01 00:00:00


In [57]:
# If you change a class variable, the default argument does not change
@dataclass
class Car:
    make: str
    model: str
    year: int = 2019
    production_date: datetime = field(default=datetime(2019, 1, 1))
    
Car.year = 2016
print(Car.year)
car = Car(make='chevy', model='sonic')
print(car)

2016
Car(make='chevy', model='sonic', year=2019, production_date=datetime.datetime(2019, 1, 1, 0, 0))


In [58]:
# If you want the default argument to change with the class variable,
# you can use default_factor
def default_year():
    return Car.year

@dataclass
class Car:
    make: str
    model: str
    year: int = field(default_factory=default_year)
    production_date: datetime = field(default=datetime(2019, 1, 1))
    
Car.year = 2016
print(Car.year)
car = Car(make='chevy', model='sonic')
print(car)

2016
Car(make='chevy', model='sonic', year=2016, production_date=datetime.datetime(2019, 1, 1, 0, 0))


In [52]:
# If you want to do something in the __init__ without have to define your own,
# you can use __post_init__
@dataclass
class Car:
    make: str
    model: str
    year: int = 2019
    last_service: datetime = field(default_factory=datetime.now)
    production_date: datetime = field(default=datetime(2019, 1, 1))

    def __post_init__(self):
        assert self.last_service >= self.production_date

car = Car(make='chevy', model='sonic', year=2016, last_service=datetime(2018, 1, 1))

AssertionError: 